In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
KEY = os.getenv("OPENAI_API_KEY")

In [6]:
llm = ChatOpenAI(
    openai_api_key=KEY,
    model="gpt-4o",
    temperature=0.5
)

C:\Users\jaggi\AppData\Local\Temp\ipykernel_7916\2624823570.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },

    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },

    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [9]:
TEMPLATE = """
You are an expert question generator.

Generate {number} multiple-choice questions from the following text.

Text:
{text}

Subject: {subject}
Tone: {tone}

Return the questions in the following JSON format:
{response_json}

Rules:
- Each question should be clear and concise.
- Provide 4 options per question (A, B, C, D).
- Mark the correct option.
- Ensure questions cover key concepts from the text.
- Do not repeat questions.

Begin.
"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose=True)

C:\Users\jaggi\AppData\Local\Temp\ipykernel_7916\343908629.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose=True)


In [12]:
TEMPLATE_2 = """
You are an expert English grammarian and writer. Given a multiple-choice quiz designed for {subject} students, evaluate the complexity of the questions and provide a complete analysis of the quiz. Use a maximum of 50 words to comment on complexity. 

If the quiz does not align with the cognitive and analytical abilities of the students, revise the questions that need improvement. Adjust the tone and language to ensure it perfectly matches the students' comprehension level.

Quiz_MCQs: {quiz}

Provide your expert analysis of the above quiz:
"""

In [13]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = TEMPLATE_2
)

In [14]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose=True)

In [15]:
quiz_review_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"], output_variables = ["quiz", "review"], verbose=True)

In [16]:
file_path = r"C:\Users\jaggi\Desktop\McqGeneratorProject\data.txt"

In [17]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [18]:
print(TEXT)

Machine Learning (ML) is a subfield of artificial intelligence (AI) that focuses on developing algorithms that allow computers to learn from and make decisions based on data, without being explicitly programmed for every specific task. The core idea is that a system can improve its performance on a task over time as it is exposed to more data. The origins of machine learning trace back to the early 1950s, when computer scientist Arthur Samuel created a program that learned to play checkers, widely considered the first self-learning program. Around the same time, Alan Turing introduced the concept of machine intelligence with his famous Turing Test. In 1957, Frank Rosenblatt developed the "Perceptron," the first neural network algorithm, inspired by the functioning of the human brain. Over the following decades, researchers developed key methods like the nearest neighbor algorithm and backpropagation for training multi-layer neural networks. However, progress was slow due to limited com

In [19]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "Academic"
RESPONSE_JSON = json.dumps(RESPONSE_JSON)

In [20]:
with get_openai_callback() as cb:
    response = quiz_review_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": RESPONSE_JSON
        }
    )

C:\Users\jaggi\AppData\Local\Temp\ipykernel_7916\3023380850.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = quiz_review_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert question generator.

Generate 5 multiple-choice questions from the following text.

Text:
Machine Learning (ML) is a subfield of artificial intelligence (AI) that focuses on developing algorithms that allow computers to learn from and make decisions based on data, without being explicitly programmed for every specific task. The core idea is that a system can improve its performance on a task over time as it is exposed to more data. The origins of machine learning trace back to the early 1950s, when computer scientist Arthur Samuel created a program that learned to play checkers, widely considered the first self-learning program. Around the same time, Alan Turing introduced the concept of machine intelligence with his famous Turing Test. In 1957, Frank Rosenblatt developed the "Perceptron," the first neural network algorithm, inspired by the functioning of the human b

In [21]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Token:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost: ${cb.total_cost}")

Total Tokens: 2022
Prompt Token:1548
Completion Tokens:474
Total Cost: $0.00861


In [22]:
quiz = response.get("quiz")

In [23]:
quiz

'```json\n{\n    "1": {\n        "mcq": "What was the first self-learning program developed in the early 1950s?",\n        "options": {\n            "a": "AlphaGo",\n            "b": "Deep Belief Networks",\n            "c": "Checkers program by Arthur Samuel",\n            "d": "Perceptron by Frank Rosenblatt"\n        },\n        "correct": "c"\n    },\n    "2": {\n        "mcq": "Which algorithm marked the beginning of the \'deep learning\' era in 2006?",\n        "options": {\n            "a": "Support Vector Machines",\n            "b": "Deep Belief Networks",\n            "c": "Random Forests",\n            "d": "Nearest Neighbor Algorithm"\n        },\n        "correct": "b"\n    },\n    "3": {\n        "mcq": "What is the primary focus of supervised learning in machine learning?",\n        "options": {\n            "a": "Discovering patterns without labels",\n            "b": "Training algorithms on labeled data",\n            "c": "Learning from rewards and penalties",\n      

In [24]:
cleaned_quiz = quiz.replace("```json", "").replace("```", "").strip()

In [25]:
quiz_dict = json.loads(cleaned_quiz)

In [26]:
quiz_json_output = json.dumps(quiz_dict, indent=4)

In [29]:
print(quiz_json_output)

{
    "1": {
        "mcq": "What was the first self-learning program developed in the early 1950s?",
        "options": {
            "a": "AlphaGo",
            "b": "Deep Belief Networks",
            "c": "Checkers program by Arthur Samuel",
            "d": "Perceptron by Frank Rosenblatt"
        },
        "correct": "c"
    },
    "2": {
        "mcq": "Which algorithm marked the beginning of the 'deep learning' era in 2006?",
        "options": {
            "a": "Support Vector Machines",
            "b": "Deep Belief Networks",
            "c": "Random Forests",
            "d": "Nearest Neighbor Algorithm"
        },
        "correct": "b"
    },
    "3": {
        "mcq": "What is the primary focus of supervised learning in machine learning?",
        "options": {
            "a": "Discovering patterns without labels",
            "b": "Training algorithms on labeled data",
            "c": "Learning from rewards and penalties",
            "d": "Adapting to scenarios with 

In [30]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({
        "MCQ": mcq,
        "Options": options,
        "Correct Answer": correct
    })

In [31]:
quiz_table_data

[{'MCQ': 'What was the first self-learning program developed in the early 1950s?',
  'Options': 'a: AlphaGo | b: Deep Belief Networks | c: Checkers program by Arthur Samuel | d: Perceptron by Frank Rosenblatt',
  'Correct Answer': 'c'},
 {'MCQ': "Which algorithm marked the beginning of the 'deep learning' era in 2006?",
  'Options': 'a: Support Vector Machines | b: Deep Belief Networks | c: Random Forests | d: Nearest Neighbor Algorithm',
  'Correct Answer': 'b'},
 {'MCQ': 'What is the primary focus of supervised learning in machine learning?',
  'Options': 'a: Discovering patterns without labels | b: Training algorithms on labeled data | c: Learning from rewards and penalties | d: Adapting to scenarios with limited labels',
  'Correct Answer': 'b'},
 {'MCQ': 'Which of the following is NOT a key component of an ML system?',
  'Options': 'a: Data preprocessing | b: Feature engineering | c: Quantum computing | d: Hyperparameter tuning',
  'Correct Answer': 'c'},
 {'MCQ': 'Which fields ar

In [33]:
quiz = pd.DataFrame(quiz_table_data)

In [34]:
quiz.to_csv("machine_learning_quiz.csv", index=False)

In [3]:
from datetime import datetime
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2025-08-11 20:11:06'